# OPUS20 THIP: Launching the room climate data logger via ethernet

This is a application notebook to query the temperature / humidity / air pressure logging device [OPUS20 THIP](https://www.lufft.com/products/in-room-measurements-291/opus-20-thip-1983/) produced by Lufft.

![OPUS20 THIP by Lufft](images/opus20-thip_climatelogger.jpg)  
OPUS20 THIP by Lufft

To read the climate data from the OPUS20 the Python package **opus20** is used. This can be found here: [https://github.com/pklaus/opus20](https://github.com/pklaus/opus20).

## Installation of required Python libraries

The installation takes place in a Python environment:
```
$ source ~/jupyter-env/bin/activate
$ pip install opus20
```

## Test function to verify the successful installation of the OPUS20

Import class **opus20** and connect to OPUS20 via it's IP:

In [2]:
from opus20 import Opus20, OPUS20_CHANNEL_SPEC, Opus20ConnectionException

try:
    o20 = Opus20('192.168.10.157')
    
except Opus20ConnectionException as e:
    str(e)

Get all available logger channels:

In [3]:
for channel in o20.available_channels:
    log_enabled = o20.get_channel_logging_state(channel)
    log_enabled = 'yes' if log_enabled else 'no'
    fmt = "Channel {:5d} (0x{:04X}): {name:22s}  unit: {unit:4s}  offset: {offset:5s}  logging: {log_enabled}"
    print(fmt.format(channel, channel, log_enabled=log_enabled, **OPUS20_CHANNEL_SPEC[channel]))

Channel   100 (0x0064): CUR temperature         unit: °C    offset: ±10.0  logging: no
Channel   120 (0x0078): MIN temperature         unit: °C    offset: ±10.0  logging: no
Channel   140 (0x008C): MAX temperature         unit: °C    offset: ±10.0  logging: no
Channel   160 (0x00A0): AVG temperature         unit: °C    offset: ±10.0  logging: yes
Channel   105 (0x0069): CUR temperature         unit: °F    offset: 0.0    logging: no
Channel   125 (0x007D): MIN temperature         unit: °F    offset: 0.0    logging: no
Channel   145 (0x0091): MAX temperature         unit: °F    offset: 0.0    logging: no
Channel   165 (0x00A5): AVG temperature         unit: °F    offset: 0.0    logging: no
Channel   200 (0x00C8): CUR relative humidity   unit: %     offset: ±30.0  logging: no
Channel   220 (0x00DC): MIN relative humidity   unit: %     offset: ±30.0  logging: no
Channel   240 (0x00F0): MAX relative humidity   unit: %     offset: ±30.0  logging: no
Channel   260 (0x0104): AVG relative humid

In [4]:
import time

channels_dict = { 'CHAN_TEMP_CURR':  100,
                  'CHAN_HUM_CURR':   200,
                  'CHAN_PRESS_CURR': 300 }

values_arr = [0, 0, 0]

idx = 0
for channel_val in o20.multi_channel_value(channels_dict.values()):
    #print("{:.3f}".format(channel_val))
    values_arr[idx] = channel_val
    idx += 1
    
print("<{time:s}> Temperature: {temperature:.3f} °C, Humidity: {humidity:.3f} % rH, Pressure: {pressure:.3f} hPa"
      .format(time=time.strftime('%Y-%m-%d %H:%M:%S'), 
              temperature=values_arr[0], 
              humidity=values_arr[1],
              pressure=values_arr[2] ))

<2022-05-25 09:30:55> Temperature: 23.333 °C, Humidity: 42.438 % rH, Pressure: 1001.450 hPa


In [5]:
temp_o20 = o20.multi_channel_value([100])

In [6]:
temp_o20[0]

23.333229064941406

The OPUS20 takes it very badly if the connection is not disconnected cleanly ... so do it everytime!

In [8]:
o20.disconnect()

In [9]:
from opus20 import Opus20, OPUS20_CHANNEL_SPEC
import time

OPUS20_IP = '192.168.10.157'

o20 = Opus20(OPUS20_IP)

INTERVAL = 1.0

channels_dict = { 'CHAN_TEMP_CURR':  100,
                  'CHAN_HUM_CURR':   200,
                  'CHAN_PRESS_CURR': 300 }

values_arr = [0, 0, 0]

while True:
    try:
        idx = 0
        for channel_val in o20.multi_channel_value(channels_dict.values()):
            #print("{:.3f}".format(channel_val))
            values_arr[idx] = channel_val
            idx += 1

        print("<{time:s}> Temperature: {temperature:.2f} °C, Humidity: {humidity:.2f} % rH, Pressure: {pressure:.2f} hPa"
              .format(time=time.strftime('%Y-%m-%d %H:%M:%S'), 
                      temperature=values_arr[0], 
                      humidity=values_arr[1],
                      pressure=values_arr[2] ))
        
        time.sleep(INTERVAL)

    except:
        # The OPUS20 takes it very badly if the connection is not disconnected cleanly ...
        o20.disconnect()
        print("Keyboard Interrupt ^C detected.")
        print("Bye.")
        break

<2022-05-25 09:31:41> Temperature: 23.33 °C, Humidity: 42.39 % rH, Pressure: 1001.46 hPa
<2022-05-25 09:31:42> Temperature: 23.33 °C, Humidity: 42.39 % rH, Pressure: 1001.46 hPa
<2022-05-25 09:31:43> Temperature: 23.33 °C, Humidity: 42.39 % rH, Pressure: 1001.46 hPa
<2022-05-25 09:31:44> Temperature: 23.33 °C, Humidity: 42.39 % rH, Pressure: 1001.46 hPa
<2022-05-25 09:31:45> Temperature: 23.33 °C, Humidity: 42.39 % rH, Pressure: 1001.46 hPa
<2022-05-25 09:31:46> Temperature: 23.33 °C, Humidity: 42.39 % rH, Pressure: 1001.46 hPa
<2022-05-25 09:31:47> Temperature: 23.33 °C, Humidity: 42.39 % rH, Pressure: 1001.46 hPa
<2022-05-25 09:31:48> Temperature: 23.33 °C, Humidity: 42.39 % rH, Pressure: 1001.46 hPa
<2022-05-25 09:31:49> Temperature: 23.33 °C, Humidity: 42.39 % rH, Pressure: 1001.46 hPa
<2022-05-25 09:31:50> Temperature: 23.33 °C, Humidity: 42.39 % rH, Pressure: 1001.46 hPa
<2022-05-25 09:31:51> Temperature: 23.33 °C, Humidity: 42.39 % rH, Pressure: 1001.46 hPa
<2022-05-25 09:31:52>